<a href="https://colab.research.google.com/github/husnain-ali21/transformers/blob/main/Vanilla_Vision_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd ./drive/MyDrive/Colab\ Notebooks

import torch 
import numpy as np 
from einops import rearrange
from torch.nn import functional as F
from torch import nn



/content/drive/MyDrive/Colab Notebooks


In [2]:
class MultiHeadAttention(nn.Module):
    def __init__(self , heads , embedding_dim , sequence_size , batch_dim ):
        super(SelfAttention ,self).__init__() 
        self.heads = heads 
        self.embedding_dim = embedding_dim 
        self.sequence_size = sequence_size
        self.batch_dim = batch_dim

        self.transform_to_keys = nn.Linear(embedding_dim , embedding_dim * heads)
        self.transform_to_queries = nn.Linear(embedding_dim , embedding_dim*heads)
        self.transform_to_values = nn.Linear(embedding_dim , embedding_dim * heads)

        self.final = nn.Linear(heads*embedding_dim  , embedding_dim)

        def forward(self , input ,  mask = None):
            assert input.shape == (self.batch_dim ,self.sequence_size , self.embedding_dim) , "Input shape is not (b x s x e )"
            queries = einops.rearrange(self.transform_to_queries(input) , \
                                       'batch seq_size (heads emb_dim) -> (batch heads) seq_size emb_dim ' , heads = self.heads)
            keys = einops.rearrange(self.transform_to_keys(input) , \
                                   ' batch seq_size (heads emb_dim) -> (batch heads) seq_size emb_dim ' , heads = self.heads)
            values = einops.rearrange(self.transform_to_values(input) ,\
                                      'batch seq_size (heads emb_dim) -> (batch heads) seq_size emb_dim ' , heads = self.heads)
            dot_product = torch.einsum("bse,bte->bst" , [keys ,queries]) / np.sqrt(self.embedding_dim)
            if mask is not None :
                dot_product = dot_product.masked_fill(mask[None] , -np.inf)
            attention_weights = torch.nn.functional.softmax(dot_product , dim =-1)
            attention_applied_output = torch.einsum('bst,bte->bse',[attention_weights , values]) # b here actually is b*h 

            return self.final(einops.rearrange(attention_applied_output ,\
                                               "(batch heads) seq_size emb_size ->  batch seq_size (heads emb_dim)" , heads = self.heads))




In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self  ,ffnn_dim , heads , embedding_dim , sequence_size , batch_dim):
        super(TransformerBlock , self).__init__() 
        self.heads = heads 
        self.embedding_dim = embedding_dim 
        self.sequence_size = sequence_size
        self.batch_dim = batch_dim

        self.norm1 = nn.LayerNorm()
        self.norm2 = nn.LayerNorm()

        self.MHA = MultiHeadAttention(self.heads , self.embedding_dim , self.sequence_size ,self.batch_dim)
        self.drop = nn.Dropout(0.3)
        self.ffnn = nn.Sequential(
            nn.Linear(embedding_dim , ffnn_dim) ,nn.LeakyReLU(0.3) , nn.Dropout(dropout) , 
            nn.Linear(ffnn ,embedding_dim) ,nn.LeakyReLU(0.3) , nn.Dropout(dropout) 
        )
        def forward(self , input , mask = None):
            output = self.norm1(self.drop(self.MHA(input , mask )) + input)
            return self.norm2(self.ffnn(output) + output)


In [ ]:
class Transformer(nn.Module):
    def __init__(self ,n_blocks , ffnn_dim , heads , embedding_dim , sequence_size , batch_dim)):
        super(Transformer , self).__init__()
        self.blocks = [TransformerBlock(ffnn_dim , heads , embedding_dim , sequence_size , batch_dim) for _ in range(n_blocks))]
        self.layers = nn.Sequential(self.blocks)

        def forward(self , input , mask) = None:
            return self.layers(input)

            


In [ ]:
class VisionTransformer(nn.Module):
    def __init__(self , image_dim , channels_in=3 , patch_size=16 , patch_dim , blocks=6 , heads=4 , linear_block_dim=1024,
                 drop = 0.2  ,p_dim):
        """
        Parameters .

        image_dim: image_size C x H x W
        channels_in: no of channels of image input (C)
        patch_size: patch dim to divide image in
        patch_dim: dim of linear layer to project patches in a low dimensional space
        blocks: no of transformer encoder blocks
        heads: not of Multi Head Attention Heads
        linear_block_dim: transformer's linear block's dim
        
        """

        super(VisionTransformer,self)__init__()
        assert image_dim % patch_size == 0 ,f"Patch size {patch_size} is not divisible."

        self.patch_dim = patched_dim
        self.patch_size = patch_size
        self.blocks = block


In [ ]:
print(patched_images.shape)

torch.Size([1, 4, 768])


In [ ]:
from einops import rearrange
p = 16 # P in maths
x_p = rearrange(t1, 'b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = p, p2 = p)

In [ ]:
print(x_p.shape)

torch.Size([1, 4, 768])


In [ ]:
(32*32 )/(16**2)

4.0